In [1]:
import torch
import torch.optim as opt
import torch.nn as nn

In [2]:
from utils import DataTwoDim

In [3]:
data = DataTwoDim()

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.first_conv_layer = nn.Conv2d(1, 10, kernel_size=3)
        self.first_activation = nn.ReLU()
        self.second_layer = nn.LazyLinear(16)
        self.second_activation = nn.ReLU()
        self.third_layer = nn.Linear(16, 16)
        self.third_activation = nn.ReLU()
        self.out_layer = nn.Linear(16, 10)
        self.out_activation = nn.Softmax(dim=1)

    def forward(self, t: torch.Tensor):
        out = self.first_activation(self.first_conv_layer(t))
        dim = 1
        for i in range(1, len(out.shape)):
            dim *= out.shape[i]
        out = out.reshape(len(t), dim)
        out = self.second_activation(self.second_layer(out))
        out = self.third_activation(self.third_layer(out))
        out = self.out_activation(self.out_layer(out))
        return out

In [ ]:
model = Model()

In [ ]:
preds = model(data.training_images)


In [ ]:
preds.shape

In [ ]:
optimizer = opt.Adam(model.parameters(), lr=1e-3)

In [ ]:
loss_fn = nn.CrossEntropyLoss()

In [4]:
from mnist import MNIST
mnist_data = MNIST('samples')
training_images, training_labels = mnist_data.load_training()

self_training_images = torch.tensor(training_images).type(torch.FloatTensor) / 255
self_training_images = torch.reshape(self_training_images, (len(training_images), 28, 28)).unsqueeze(1)
self_training_labels = torch.tensor(training_labels)

_, test_labels = mnist_data.load_testing()
self_test_labels = torch.tensor(test_labels)

In [ ]:
for epoch in range(1, 501):
    prediction = model(data.training_images)
    loss = loss_fn(prediction, self_training_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0 or epoch == 1:
        print(f"{epoch} : {loss.item()}")

In [ ]:
torch.save(model, "cnnmodelone.model")

In [ ]:
test_preds = model(data.test_images)

In [ ]:
loss = loss_fn(test_preds, self_test_labels)
loss.item()

In [ ]:
for i in range(100):
    pred = test_preds[i]
    maxout = max(pred)
    for j in range(10):
        if pred[j] == maxout:
            print(f"{j} : {pred[j]} : {self_test_labels[i]}")

In [2]:
class ModelTwo(nn.Module):
    def __init__(self):
        super().__init__()
        self.first_conv_layer = nn.Conv2d(1, 16, kernel_size=5)
        self.first_activation = nn.ReLU()
        self.second_layer = nn.LazyLinear(32)
        self.second_activation = nn.ReLU()
        self.third_layer = nn.Linear(32, 16)
        self.third_activation = nn.ReLU()
        self.out_layer = nn.Linear(16, 10)
        self.out_activation = nn.Softmax(dim=1)

    def forward(self, t: torch.Tensor):
        out = self.first_activation(self.first_conv_layer(t))
        dim = 1
        for i in range(1, len(out.shape)):
            dim *= out.shape[i]
        out = out.reshape(len(t), dim)
        out = self.second_activation(self.second_layer(out))
        out = self.third_activation(self.third_layer(out))
        out = self.out_activation(self.out_layer(out))
        return out

In [ ]:
model_two = ModelTwo()

In [5]:
optimizer_two = opt.Adam(model_two.parameters(), lr=1e-2)
loss_fn_two = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(1, 501):
    prediction = model_two(data.training_images)
    loss = loss_fn_two(prediction, self_training_labels)
    optimizer_two.zero_grad()
    loss.backward()
    optimizer_two.step()
    if epoch % 10 == 0 or epoch == 1:
        print(f"{epoch} : {loss.item()}")

In [ ]:
torch.save(model_two, "cnnmodeltwo.model")

In [ ]:
test_preds_two = model_two(data.test_images)

In [ ]:
loss_two = loss_fn_two(test_preds_two, self_test_labels)
loss_two.item()

In [ ]:
samples, error_count = 0, 0
for i in range(len(self_test_labels)):
    pred = test_preds_two[i]
    maxout = max(pred)
    for j in range(10):
        if pred[j] == maxout:
            if j != self_test_labels[i]:
                error_count += 1
    samples += 1
print(f"{samples} : {error_count}")

In [3]:
model_loaded = torch.load("cnnmodeltwo.model")

In [8]:
training_images_grad = data.training_images
training_images_grad.requires_grad = True

In [9]:
output = model_loaded(training_images_grad)

In [13]:
loss = loss_fn_two(output, data.training_labels)

In [15]:
loss.backward()

In [21]:
grad = training_images_grad.retain_grad()

In [22]:
type(grad)

NoneType

In [9]:
from mnist import MNIST
mnist_data = MNIST('samples')
training_images, training_labels = mnist_data.load_training()

original_images = torch.tensor(training_images)
self_training_images = original_images.type(torch.FloatTensor) / 255
self_training_images = torch.reshape(self_training_images, (len(training_images), 28, 28)).unsqueeze(1)
self_training_images.requires_grad = True
self_training_labels = torch.zeros(len(training_labels), 10)
for i in range(len(training_labels)):
    self_training_labels[i, training_labels[i]] = 1
self_training_labels_simple = torch.tensor(training_labels)

In [5]:
loss = nn.MSELoss()(model_loaded(self_training_images), self_training_labels)
loss.backward()
grad = self_training_images.grad

In [6]:
loss.item()

0.0014221783494576812

In [10]:
grad

tensor([[[[ 3.8281e-21,  5.0412e-21, -3.4931e-21,  ...,  2.4893e-20,
           -1.3448e-20, -8.9814e-21],
          [ 7.1731e-21,  2.6138e-20,  3.0319e-20,  ..., -1.4123e-18,
           -1.0721e-18, -3.0685e-19],
          [ 2.5085e-20,  2.9420e-20,  5.3708e-20,  ..., -1.3069e-18,
           -5.8201e-19, -4.4732e-19],
          ...,
          [-2.9700e-18, -3.3924e-18,  9.2318e-19,  ...,  9.5666e-20,
            7.6870e-20,  3.9316e-20],
          [ 4.1032e-19, -7.6883e-19,  2.2572e-18,  ...,  5.3277e-21,
            3.5755e-20,  2.8956e-20],
          [ 2.9862e-19, -3.2152e-19, -1.2979e-18,  ..., -2.1254e-20,
            4.5560e-21,  9.4070e-21]]],


        [[[ 2.0050e-31,  4.3656e-32, -4.0506e-31,  ..., -4.0519e-31,
            7.0237e-32,  2.8119e-31],
          [ 5.0205e-31,  6.8881e-31, -7.5509e-32,  ..., -1.0354e-31,
            6.0889e-31,  7.4820e-31],
          [ 7.1306e-31,  1.3797e-30,  1.7354e-30,  ...,  1.4451e-30,
            1.3245e-30,  1.1288e-30],
          ...,
   

In [7]:
self_altered_training_images = self_training_images + 1e-1 * grad.apply_(lambda x: 1 if x >= 0 else -1)

In [8]:
loss = nn.MSELoss()(model_loaded(self_altered_training_images), self_training_labels)
loss.item()

0.11363277584314346

In [17]:
image = self_training_images[0].unsqueeze(0)
image.requires_grad = True

RuntimeError: you can only change requires_grad flags of leaf variables.

In [21]:
pred = model_loaded(image)

In [10]:
self_training_labels[0]

tensor(5)

In [22]:
loss = loss_fn_two(pred, self_training_labels[0].unsqueeze(0))

In [28]:
loss.backward()

RuntimeError: Trying to backward through the graph a second time (or directly access saved variables after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved variables after calling backward.

In [26]:
grad = image.retain_grad()

In [27]:
type(grad)

NoneType

In [19]:
image = torch.tensor(training_images[0]).type(torch.FloatTensor) / 255
image.requires_grad = True
image.shape

torch.Size([784])

In [20]:
image = torch.reshape(image, (1, 1, 28, 28))

In [10]:
def get_prediction(model, image):
    my_image = image.unsqueeze(0)
    pred = model(my_image)
    maxout = max(pred[0])
    for j in range(10):
        if pred[0, j] == maxout:
           return j, maxout

In [11]:
possible_adversarials = []
for i in range(len(self_training_images)):
    prediction, confidence = get_prediction(model_loaded, self_altered_training_images[i])
    if prediction != self_training_labels_simple[i]:
        params = {"Label": self_training_labels_simple[i], "Prediction": prediction, "Confidence": confidence,
                  "Index": i}
        possible_adversarials.append(params)

In [12]:
len(possible_adversarials)

35948

In [13]:
original = []
for i in range(len(self_training_images)):
    prediction, confidence = get_prediction(model_loaded, self_training_images[i])
    if prediction != self_training_labels_simple[i]:
        params = {"Label": self_training_labels_simple[i], "Prediction": prediction, "Confidence": confidence,
                  "Index": i}
        original.append(params)
print(len(original))

443


In [24]:
def display( image, width=28, threshold=200, is_mask=False):
    img = image[0]
    img = torch.reshape(img, (28 * 28,))
    render = ''
    if is_mask:
        for i in range(len(img)):
            if i % width == 0:
                render += '\n'
            if img[i] > threshold:
                render += '+'
            elif img[i] < - threshold:
                render += '-'
            else:
                render += '.'
        return render
    for i in range(len(img)):
        if i % width == 0:
            render += '\n'
        if img[i] > threshold:
            render += '@'
        else:
            render += '.'
    return render

In [29]:
adversarial = possible_adversarials[0]
original_image = self_training_images[adversarial["Index"]]
altered_image = self_altered_training_images[adversarial["Index"]]
mask = altered_image - original_image
mask = mask * 255
original_image = original_image * 255
altered_image = original_image * 255
print(display(original_image))
print("+++++")
print(display(mask, is_mask=True, threshold=25))
print("=====")
print(display(altered_image, threshold=150))
print(f"{adversarial['Label']} : {adversarial['Prediction']} : {adversarial['Confidence']}")


............................
............................
............................
............................
............................
.....................@@.....
.............@@@@@@.@@......
........@@@@@@@@@@..........
........@@@@@@..@@..........
...........@@@..............
............@...............
............@...............
.............@..............
.............@@.............
..............@@@...........
................@@..........
.................@@.........
.................@@@........
.................@@@........
..............@@@@@.........
............@@@@@@..........
..........@@@@@.............
........@@@@@...............
......@@@@@@................
.....@@@@...................
............................
............................
............................
+++++

++-----+--+++++++--+++-+++--
+++++-++-++-----+-----------
+++++++++++++++++-----------
+++-++++++++++++---++--+++++
++++--+++-++++++---+---+++++
++++--------+++-----------++
++++++